In [1]:
import requests
import pandas as pd
import datetime
from datetime import datetime
import numpy as np

In [2]:
url_groupdetail = "https://api.purpleair.com/v1/groups/1662"
url_newmember = "https://api.purpleair.com/v1/groups/1662/members"
url_sensordata = "https://api.purpleair.com/v1/sensors/39355"
url_groupdata = "https://api.purpleair.com/v1/groups/1662/members"
url_deletemember = "https://api.purpleair.com/v1/groups/1662/members/158862"

key_get = "C111944A-B3A7-11ED-B6F4-42010A800007"
key_post = "C112A231-B3A7-11ED-B6F4-42010A800007"
headers = {'X-API-Key': key_get}

### List of sensors in San Pedro

In [3]:
sensors = requests.get(url_groupdetail, headers=headers)

In [4]:
sensors = sensors.json()

In [5]:
sensors = sensors['members']

In [6]:
sensors = pd.DataFrame(sensors)

In [7]:
len(sensors)

32

In [8]:
sensors = sensors[['sensor_index', 'id', 'created']]

In [9]:
sensors.rename(columns = {'id': 'group_id'}, inplace = True)

In [10]:
sensors = sensors.drop('created', axis=1)

In [11]:
sensors.head()

,sensor_index,group_id
0,34641,158850
1,37747,158851
2,38537,158849
3,143372,158854
4,39355,160198


### Call data - PM 2.5

In [12]:
payload_pm25 = {'fields': 'pm2.5'}

In [13]:
pm25 = requests.get(url_groupdata, headers = headers, params = payload_pm25)

In [14]:
pm25 = pm25.json()

In [15]:
pm25 = pm25['data']

In [16]:
pm25 = pd.DataFrame(pm25, columns = ('sensor_index', 'pm_25'))

In [17]:
pm25.head()

,sensor_index,pm_25
0,140432,4.3
1,140440,27.0
2,143372,26.1
3,147010,23.5
4,33117,11.4


### Call data - PM 10

In [18]:
payload_pm10 = {'fields': 'pm10.0'}

In [19]:
pm10 = requests.get(url_groupdata, headers = headers, params = payload_pm10)

In [20]:
pm10 = pm10.json()

In [21]:
pm10 = pm10['data']

In [22]:
pm10 = pd.DataFrame(pm10, columns = ('sensor_index', 'pm_10'))

In [23]:
pm10.head()

,sensor_index,pm_10
0,140432,4.8
1,140440,29.5
2,143372,31.2
3,147010,25.3
4,33117,12.4


### Call data - Temperature

In [24]:
payload_temp = {'fields': 'temperature'}

In [25]:
temp = requests.get(url_groupdata, headers = headers, params = payload_temp)

In [26]:
temp = temp.json()

In [27]:
temp = temp['data']

In [28]:
temp = pd.DataFrame(temp, columns = ('sensor_index', 'temperature'))

In [29]:
temp.head()

,sensor_index,temperature
0,140432,86
1,140440,88
2,143372,88
3,147010,94
4,33117,87


### Call data - Timestamp

In [30]:
now = datetime.now()

In [31]:
timestamp = now.strftime("%d/%m/%Y %H:%M:%S")

In [32]:
timestamp = pd.Series(timestamp)

In [33]:
timestamp = pd.DataFrame(timestamp)

In [34]:
timestamp = pd.DataFrame(np.repeat(timestamp.values, 32, axis=0))

### Join data into one database

In [35]:
sensor_pm25 = pd.concat([sensors.set_index('sensor_index'),pm25.set_index('sensor_index')], axis=1, join='inner').reset_index()

In [36]:
sensor_pm25_pm10 = pd.concat([sensor_pm25.set_index('sensor_index'), pm10.set_index('sensor_index')], axis=1, join='inner').reset_index()

In [37]:
sensor_pm25_pm10_temp = pd.concat([sensor_pm25_pm10.set_index('sensor_index'), temp.set_index('sensor_index')], axis=1, join='inner').reset_index()

In [38]:
sensor_pm25_pm10_temp['date'] = timestamp

In [39]:
sensor_pm25_pm10_temp

,sensor_index,group_id,pm_25,pm_10,temperature,date
0,34641,158850,24.9,26.5,90,27/02/2023 10:32:33
1,37747,158851,23.2,30.3,85,27/02/2023 10:32:33
2,38537,158849,27.2,33.5,92,27/02/2023 10:32:33
3,143372,158854,26.1,31.2,88,27/02/2023 10:32:33
4,39355,160198,25.7,33.6,88,27/02/2023 10:32:33
5,39697,158855,25.3,28.7,88,27/02/2023 10:32:33
6,34569,158852,19.7,21.6,95,27/02/2023 10:32:33
7,39731,158853,21.0,21.6,96,27/02/2023 10:32:33
8,147010,158858,23.5,25.3,94,27/02/2023 10:32:33
9,140440,158859,27.0,29.5,88,27/02/2023 10:32:33
